In [1]:
# 모듈 임포트
import time
import board
import busio
import adafruit_ads1x15.ads1115 as ADS
from adafruit_ads1x15.analog_in import AnalogIn
import joblib
import numpy as np
import psutil
import pandas as pd
from sklearn.preprocessing import StandardScaler
import csv
from datetime import datetime

In [2]:
# 모델 및 스케일러 로드
model = joblib.load('./model/one_class_svm_model_new.joblib')
scaler = joblib.load('./model/scaler_new.joblib')

In [3]:
# I2C 및 ADC 설정
i2c = busio.I2C(board.SCL, board.SDA)
ads = ADS.ADS1115(i2c) # (아날로그 → 디지털 변환기) 객체
ads.gain = 2/3 # 증폭 배율 설정

# ADS1115 채널 선택
voltage_ch = AnalogIn(ads, ADS.P0)
current_ch = AnalogIn(ads, ADS.P1)

In [4]:
# 센서 보정값
ZMPT_offset = 2.5
ZMPT_scale = 761.8

ACS712_offset = 2.5
ACS712_sensitivity = 0.04598

In [5]:
def add_features(voltage_seq, current_seq):
    voltage = np.mean(voltage_seq)
    current = np.mean(current_seq)
    voltage_diff = np.abs(voltage_seq[-1] - voltage_seq[-2])
    current_diff = np.abs(current_seq[-1] - current_seq[-2])
    voltage_ma = np.mean(voltage_seq[-5:])
    current_ma = np.mean(current_seq[-5:])
    power = voltage * current
    power_diff = np.abs(power - (voltage_seq[-2] * current_seq[-2]))
    return [voltage, current, voltage_diff, current_diff, voltage_ma, current_ma, power, power_diff]

In [6]:
def postprocess_anomalies_realtime(anomaly_buffer, min_consecutive=8):
    if len(anomaly_buffer) < min_consecutive:
        return False
    count = 0
    for val in reversed(anomaly_buffer):
        if val == 1:
            count += 1
        else:
            break
    return count >= min_consecutive

In [71]:
# 테스트 데이터 불러오기 (센서 대체용 - 사용 안 할 경우 생략 가능)
column_names = ['timestamp', 'v_raw', 'c_raw', 'voltage', 'current']
df = pd.read_csv('./log/0519/log_ocsvm_1_L_th2.csv', names=column_names, header=None)
df = df.dropna(subset=['voltage', 'current']).reset_index(drop=True)
csv_filename = "./log/0529/svm_드릴_1.csv"

In [ ]:
try:
    print("🚀 실시간 아크 감지 시작합니다!")

    USE_SENSOR = True  # True로 설정 시 실제 센서 사용
    ANOMALY_THRESHOLD = 2

    data_buffer = []
    anomaly_buffer = []
    first_anomaly_detected = False
    anomaly_start_time = None
    start_time = time.perf_counter()
    idx = -1

    while True:
        idx += 1

        if USE_SENSOR:
            try:
                v_raw_sensor = voltage_ch.voltage
                c_raw_sensor = current_ch.voltage
                voltage = (v_raw_sensor - ZMPT_offset) * ZMPT_scale
                current = (c_raw_sensor - ACS712_offset) / ACS712_sensitivity

                prev_voltage = voltage
                prev_current = current

            except (OSError, ValueError, RuntimeError) as e:
                print(f"⚠️ 센서 오류 발생: {e}")
                print("🩹 이전 정상값을 사용합니다.")
                voltage = prev_voltage if 'prev_voltage' in locals() else 0
                current = prev_current if 'prev_current' in locals() else 0

        else:
            voltage = df['voltage'].iloc[idx]
            current = df['current'].iloc[idx]

        data_buffer.append((voltage, current))

        # CSV 기록
        with open(csv_filename, mode='a', newline='') as file:
            writer = csv.writer(file)
            timestamp = datetime.now().isoformat()
            writer.writerow([timestamp, voltage, current, 0 if 'anomaly' not in locals() else anomaly])

        if len(data_buffer) >= 8:
            voltage_seq, current_seq = zip(*data_buffer[-8:])
            features = add_features(voltage_seq, current_seq)
            features_scaled = scaler.transform([features])
            anomaly = int(model.predict(features_scaled).flatten()[0])
            anomaly = int(anomaly == -1)  # 이상이면 1, 아니면 0

            anomaly_buffer.append(anomaly)
            if len(anomaly_buffer) > ANOMALY_THRESHOLD:
                anomaly_buffer.pop(0)

            # 첫 이상 신호 시간 기록
            if anomaly == 1 and anomaly_start_time is None:
                anomaly_start_time = time.perf_counter()
            elif anomaly == 0:
                anomaly_start_time = None

            if postprocess_anomalies_realtime(anomaly_buffer, min_consecutive=ANOMALY_THRESHOLD):
                print("⚡ 아크 이상 감지!", idx)
                if not first_anomaly_detected and anomaly_start_time is not None:
                    total_elapsed = time.perf_counter() - start_time
                    actual_elapsed = time.perf_counter() - anomaly_start_time
                    print(f"⏱️ 전체 소요 시간: {total_elapsed:.6f}초")
                    print(f"⏱️ 연속 {ANOMALY_THRESHOLD}개 이상 신호까지 실제 소요 시간: {actual_elapsed:.6f}초")
                    first_anomaly_detected = True

                    # CSV 기록
                    with open(csv_filename, mode='a', newline='') as file:
                        writer = csv.writer(file)
                        timestamp = datetime.now().isoformat()
                        writer.writerow([timestamp, voltage, current, anomaly])

                    # 시스템 자원 사용 현황 출력
                    cpu_usage = psutil.cpu_percent(interval=0.0)
                    memory_usage = psutil.Process().memory_info().rss / 1024 ** 2
                    print(f"🧠 메모리: {memory_usage:.2f}MB | 🧮 CPU: {cpu_usage:.2f}% | idx: {idx}")
                    print(f"v_raw:{v_raw_sensor}, c_raw:{c_raw_sensor}, voltage:{voltage}, current:{current}")

                    break

        # 시스템 자원 사용 현황 출력
        cpu_usage = psutil.cpu_percent(interval=0.0)
        memory_usage = psutil.Process().memory_info().rss / 1024 ** 2
        print(f"🧠 메모리: {memory_usage:.2f}MB | 🧮 CPU: {cpu_usage:.2f}% | idx: {idx}")
        print(f"v_raw:{v_raw_sensor}, c_raw:{c_raw_sensor}, voltage:{voltage}, current:{current}")

        time.sleep(0.001163)  # 860Hz 간격

except KeyboardInterrupt:
    print("🛑 실시간 감지를 종료합니다.")

🚀 실시간 아크 감지 시작합니다!
🧠 메모리: 200.64MB | 🧮 CPU: 3.70% | idx: 0
v_raw:2.567625, c_raw:2.461125, voltage:51.51672500000003, current:-0.8454762940408872
🧠 메모리: 200.64MB | 🧮 CPU: 0.00% | idx: 1
v_raw:2.3668125, c_raw:2.4980625, voltage:-101.4622375, current:-0.042137886037405946
🧠 메모리: 200.64MB | 🧮 CPU: 0.00% | idx: 2
v_raw:2.898375, c_raw:2.5876875, voltage:303.48207500000007, current:1.9070791648542826
🧠 메모리: 200.64MB | 🧮 CPU: 0.00% | idx: 3
v_raw:2.1808125, c_raw:2.441625, voltage:-243.15703749999994, current:-1.2695737277076955
🧠 메모리: 200.64MB | 🧮 CPU: 10.00% | idx: 4
v_raw:2.9105625, c_raw:2.610375, voltage:312.7665125000001, current:2.400500217485861
🧠 메모리: 200.64MB | 🧮 CPU: 0.00% | idx: 5
v_raw:2.257875, c_raw:2.4163125, voltage:-184.4508250000001, current:-1.8200848194867314
🧠 메모리: 200.64MB | 🧮 CPU: 10.00% | idx: 6
v_raw:2.8625625, c_raw:2.612625, voltage:276.2001125000001, current:2.4494345367551102
🧠 메모리: 200.64MB | 🧮 CPU: 0.00% | idx: 7
v_raw:2.388, c_raw:2.42625, voltage:-85.321600